In [1]:
from bs4 import BeautifulSoup 
import pandas as pd

In [2]:
kml_file_path = 'D:\\programing\\Final Project\\EVChargePlanner\\data\\DC EV Charging Station Thailand - Piyamate Wisanuvej (1).kml'

with open(kml_file_path, 'rt', encoding='utf-8') as data:
    kml_soup = BeautifulSoup(data, 'lxml-xml')

In [3]:
folder = kml_soup.find_all("Folder")
columns = ['Name', 'Description', 'styleUrl','NameEN', 'Power', 'ChargeCoreDescription', 'Hours', 'Dealership', 'Address', 'Latitude', 'Longitude', 'Provider', 'Coordinates']
df = pd.DataFrame(columns=columns)

for provider in folder:
    Provider = provider.find("name").text
    for placemark in provider.find_all("Placemark"):
        try:
            placemark_name = placemark.find("name").text
            description = placemark.find("description").text
            if 'etc' in Provider:
                etcProvider = Provider + ':' + description.split('<br>')[-1].split(':')[-1]
            else:
                etcProvider = None
            styleUrl = placemark.find("styleUrl").text

            ExtendedData = placemark.find("ExtendedData")
            try:
                Name_EN = ExtendedData.find("Data", attrs={"name": "Name-EN"}).find("value").text
            except:
                Name_EN = None
            Power = ExtendedData.find("Data", attrs={"name": "Power"}).find("value").text
            Charge_Core_Description = ExtendedData.find("Data", attrs={"name": "Description"}).find("value").text
            try:
                Hours = ExtendedData.find("Data", attrs={"name": "Hours"}).find("value").text
            except:
                Hours = None
            try:
                Dealership = ExtendedData.find("Data", attrs={"name": "Dealership"}).find("value").text
            except:
                Dealership = None
            try:
                Address = ExtendedData.find("Data", attrs={"name": "Address"}).find("value").text
            except:
                Address = Charge_Core_Description
            Latitude = ExtendedData.find("Data", attrs={"name": "Latitude"}).find("value").text
            Longitude = ExtendedData.find("Data", attrs={"name": "Longitude"}).find("value").text

            coordinates = placemark.find("Point").find("coordinates").text
        except:
            print(placemark)
        
        # Append values to the DataFrame
        df = df._append({
            'Name' : placemark_name, 
            'Description' : description, 
            'styleUrl' : styleUrl,
            'NameEN': Name_EN,
            'Power': Power,
            'ChargeCoreDescription': Charge_Core_Description,
            'Hours': Hours,
            'Dealership' : Dealership,
            'Address': Address,
            'Latitude': Latitude,
            'Longitude': Longitude,
            'Provider': etcProvider if etcProvider else Provider,
            'Coordinates': coordinates
        }, ignore_index=True)

df.replace('', None, inplace=True)

In [4]:
df['Provider'].unique()

array(['PTT', 'PEA', 'EleX', 'Altervim', 'EA', 'etc: EVolt', 'etc: GWM',
       'etc: ', 'etc: Haup', 'etc: Tesla', 'etc: MEA', 'etc: Sharge',
       'etc: OneCharge', 'etc: Galvanic', 'MG'], dtype=object)

In [7]:
df

,Name,Description,styleUrl,NameEN,Power,ChargeCoreDescription,Hours,Dealership,Address,Latitude,Longitude,Provider,Coordinates
0,ปตท บจ.ปฐมวิวัฒน์ 2018,Name-EN: PTT Pathomwiwat 2018<br>Power: 120 kW...,#icon-1899-DB4436-labelson,PTT Pathomwiwat 2018,120 kW,ChargeCore CCSx2: 2x CCS2 (1000V 200A),None,None,32/3 ม.2 ต.เจดีย์หัก อ.เมืองราชบุรี จ.ราชบุรี ...,13.544778,99.804667,PTT,"\n 99.804667,13.544778,0\n"
1,ปตท สิชล,Name-EN: PTT Sichon<br>Power: 120 kW<br>Descri...,#icon-1899-DB4436-labelson,PTT Sichon,120 kW,ChargeCore NKR-ADC: 2x CCS2 (1000V 200A),None,None,40 ม.9 ต.สิชล อ.สิชล นครศรีธรรมราช 80120,9.003393696,99.8816567,PTT,"\n 99.8816567,9.003393696,0\n ..."
2,ปตท บจ.พงษ์กิต 2 (หัวทะเล),"Name-EN: PTT Pongkit Co., Ltd. Branch 2 (Hua T...",#icon-1899-DB4436-labelson,"PTT Pongkit Co., Ltd. Branch 2 (Hua Talay)",120 kW,"ChargeCore NKR-ADC: 2x CCS2, Type 2 (1000V 200A)",None,None,สาขา 2 133 ม.1 ถ.ราชสีมา-โชคชัย ต.หนองบัวศาลา ...,14.9389399815351,102.132905062155,PTT,"\n 102.132905062155,14.938939981535..."
3,โรบินสัน สไตล์ สมุทรปราการ,Name-EN: Robinson Lifestyle Samut Prakan<br>Po...,#icon-1899-DB4436-labelson,Robinson Lifestyle Samut Prakan,120 kW,"ChargeCore NKR: 2x CCS2, Type 2 (1000V 200A)",None,None,789 หมู่ 2 ถนนสุขุมวิท ตำบลท้ายบ้านใหม่ อำเภอเ...,13.5838097086702,100.609540438793,PTT,"\n 100.609540438793,13.583809708670..."
4,ปตท กนิษฐาพาณิชย์,Name-EN: PTT NGV Station Kanittha Panich<br>Po...,#icon-1899-DB4436-labelson,PTT NGV Station Kanittha Panich,120 kW,ChargeCore CCSx2: 2x CCS2 (1000V 200A)\nCharge...,None,None,160/6 หมู่ที่ 2 ต.สวนใหญ่ อ.เมืองนนทบุรี จ.นนท...,13.8503351,100.4906831,PTT,"\n 100.4906831,13.8503351,0\n ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,ฺบางจาก สายไหม 63,Power: <br>Dealership: Non-dealership<br>Descr...,#icon-1899-DB4436-labelson,None,None,None,None,Non-dealership,529 หมู่ 1 ถ. สายไหม แขวงสายไหม เขตสายไหม กรุง...,13.92145,100.67558,MG,"\n 100.67558,13.92145,0\n"
1953,บางจาก ปากน้ำ,Power: <br>Dealership: Non-dealership<br>Descr...,#icon-1899-DB4436-labelson,None,None,None,None,Non-dealership,125/2 ถนนสุขุมวิท ตำบลปากน้ำ อำเภอเมืองสมุทรปร...,13.596901,100.604198,MG,"\n 100.604198,13.596901,0\n"
1954,บางจาก พหลโยธิน กม.26,Power: <br>Dealership: Non-dealership<br>Descr...,#icon-1899-DB4436-labelson,None,None,None,None,Non-dealership,พหลโยธิน 73 (ซอยหงวน เหวียนระวิ) แขวงสนามบิน เ...,13.936793,100.62448,MG,"\n 100.62448,13.936793,0\n"
1955,MG ออโต้เฮาส์ สระบุรี,Power: <br>Dealership: Dealership<br>Descripti...,#icon-1899-DB4436-labelson,None,None,None,None,Dealership,111 หมู่ที่ 1 ตำบลคำพราน อำเภอวังม่วง จังหวัดส...,14.549055,100.938756,MG,"\n 100.938756,14.549055,0\n"


In [6]:
df.isna().sum()

Name                        0
Description                 0
styleUrl                    0
NameEN                   1129
Power                     149
ChargeCoreDescription     163
Hours                    1596
Dealership               1822
Address                   110
Latitude                    0
Longitude                   0
Provider                    0
Coordinates                 0
dtype: int64